In [1]:
import pandas as pd
import numpy as np
import torch
import re
import glob
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
z = {}

In [26]:
fn

'reason-dataset/reason/abortion/P10.data.rsn'

In [44]:
data = []

topics = ['abortion', 'gayRights', 'marijuana', 'obama']
for t in topics:
    for fn in glob.glob('reason-dataset/reason/' + t + '/*.rsn'):
        with open(fn, 'r', errors='ignore') as f:
            lines = f.readlines()
        x = re.search(r'/([a-zA-Z0-9]*)\.', fn)
        src = x.group(1)
        lab_ix = -3
        d = [t, src]
        for ix, l in enumerate(lines):
            if l.startswith('Label##'):
                l = l[len('Label##'):].rstrip()
                d.append(l)
                lab_ix = ix
            if ix - 1 == lab_ix:
                l = l[len('Line##'):].rstrip()
                d.append(l)
                data.append(d)
                d = [t, src]


In [67]:
df = pd.DataFrame(data, columns=['topic', 'source', 'substance', 'line'])
df['stance'] = df['substance'].apply(lambda s: 'Pro' if s[0] == 'p' else 'Con')
df.index.name = 'id'
df = df[['topic', 'stance', 'substance', 'source', 'line']]

In [69]:
df.to_csv('reason-dataset.csv')

In [65]:
df

,topic,source,substance,line,stance
id,,,,,
0,abortion,Q34,c-adopt,And if it is not possible for your to have a b...,Con
1,abortion,Q34,c-kill,I believe that abortion cannot be justified be...,Con
2,abortion,Q34,c-baby_right,"Moreover, United Nations declaration says chil...",Con
3,abortion,M25,c-baby_right,"Even if it doesn't have a brain, my belief is ...",Con
4,abortion,O43,c-sex,"Can't you use a condom while having sex. Yes, ...",Con
...,...,...,...,...,...
2843,obama,B64,p-quality,"While I wish he would have accomplished more, ...",Pro
2844,obama,B64,p-republicans,Obama inherited many of the deficiencies lefto...,Pro
2845,obama,F38,c-War,He has gotten troops out of countries Bush...,Con


In [36]:
for ix, x in enumerate(pairs):
    lab = x['label']
    src = x['src']
    l = x['line']
    print(ix, l)

0 And if it is not possible for your to have a baby, due to shortage of money or time... Nowadays many couples would adopt unwanted babies.
1 I believe that abortion cannot be justified because it is killing an innocent human being.
2 Moreover, United Nations declaration says children have a right to protected before as well as after birth!
3 Even if it doesn't have a brain, my belief is that the PERSON still has a soul. The soul is what gives the PERSON LIFE
4 Can't you use a condom while having sex. Yes, having the baby puts the teen mother in a bad spot, but it's her fault for getting pregnant in the first place.
5 to abort a child is killing the child.
6 From the time of conception, a child is alive
7 If its taking nourishment from its mother, it's not quite a human being yet.
8 I simply think it's the lesser evil of two, especially when you look at statistics that show that abortion laws don't stop abortions, they just stop safe abortions. This leads to major risks to both the bab

In [13]:
ixlis = []
for ix, l in enumerate(lines):
    if l.startswith('Label##'):
        print(l)
        ixlis.append(ix)
    if ix - 1 in ixlis:
        print(l)

Label##c-adopt

Line##And if it is not possible for your to have a baby, due to shortage of money or time... Nowadays many couples would adopt unwanted babies.

Label##c-kill

Line##I believe that abortion cannot be justified because it is killing an innocent human being.

Label##c-baby_right

Line##Moreover, United Nations declaration says children have a right to protected before as well as after birth!



In [5]:
pas = {
    'Adolescent Rights': ['Many adolescents cannot make responsible decisions', 'Adolescents  are  as capable as adults'],
    'Animal Rights': ['Animals  should  not  be treated as property', 'There is nothing wrong with  using  animals  to further human interests'],
    'Big government': ['Public   utility   is   best served by actions coordinated by central government', 'Public  interest  is  best served and propelled by voluntary   interactions,and not ones dictated by government'],
    'Black market': ['Prohibiting products and    activities    makes them  less   visible  and available, and thus less harmful', 'Prohibition  is  counterproductive and only leads to increased demand'],
    'Clean energy': ['Humanity    must    embrace  clean  energy  in order  to  fight  climate change', 'Ecological concerns add   further   strain   on the economy'],
    'Coercion': ['A decisive and enforced policy is the best way to deliver a message', 'Enforcement   tends   to be   less   effective   than persuasion  and  education'],
    'Conservatism': ['The  current  system  is working,   and   making such   a   change   could have   negative   consequences', 'It is time to change the old ways and try something new'],
    'Criminal deterrence': ['When people will have to pay for their actions there will be less crime', 'Strict punishment is not effective  in  preventing criminal behavior'],
    'Discrimination': ['It  is  a  fact  that  there are differences between people. Hence,  there should   sometimes   be differences  in  the  way people are treated.', 'All   people   should   be treated equally'],
    'Environment': ['People must protect nature and respect its biological communities', 'Environmentalism stands  in  the  way  of technological   progress and economic growth'],
    'Fair trial': ['Upholding the rights of the  accused  and  ensuring a fair process is the best   way   to   maintain justice', 'The focus should not be on  the  rights  of  criminals,  but  on  protecting the  law-abiding  public from harm'],
    'Fixable': ['There  are  some  issues here that need addressing,   but   that   doesn’t mean  it  should  all  be eliminated', 'Due to the many problems associated with [TOPIC], the best course of action would be to put an end to it'],
    'Framework': ['[TOPIC] works efficiently', '[TOPIC] fails to achieve its goals'],
    'Freedom of choice': ['People  have  the  right to make their own choices,  including  bad ones', 'It is the duty of society to  protect  people  from their own bad choices'],
    'Freedom of religion': ['People should be free to practice their religion', 'Religions are outdated, irrational and harmful'],
    'Gender equality': ['Banishing the misguided notions of gender  roles  in  society is  the  way  to  achieve true equality for women', 'Gender roles reflect true biological differences   between   the sexes.   It doesn’t make sense to ignore them'],
    'Greater good': ['The   safety   and   wellbeing of the community is  more  important  than individual freedom', 'Individual freedom is a sacred value.  It cannot be subordinated to subjective  opinions  deciding what is best for society'],
    'Immigration': ['People   who   come   in search  of  a  safer  and better life should not be turned away', 'Mass immigration threatens social cohesion'],
    'Media’s impact': ['Media consumption has no  significant  social  or behavioral effects', 'Stereotypes  distributed in  the  media  lead  to  a distorted  view  of  society and of the other'],
    'National security': ['Some  rights  and  freedoms  need  to  be  limited  in  the  interest  of national security', 'Security does not justify   brushing   aside fundamental  rights  and freedoms'],
    'Objectification': ['Women should have the power to use and show their   bodies   as   they would like to', 'Society   cannot   allow women to be treated as commodities'],
    'Offensive speech': ['Freedom  of  expression is meaningless if it does not  apply  to  troubling and controversial ideas', 'The freedom of expression does not legitimize offending  people’s  values and beliefs'],
    'Playing god': ['People  can,  and  therefore   should,   interfere with  nature  in  order  to take care of their needs', 'Only  God  should  determine how life comes into  being  and  how  it comes to an end'],
    'Price interference': ['Price regulation is useful for achieving social and economic goals', 'Market   forces   should determine  the  rates  of prices and fees'],
    'Privatization': ['Privatization often leads to improved efficiency and quality', 'The state is a better and a more natural provider of public goods and services'],
    'Public health': ['It  is  the  government’s duty  to  safeguard  public  health  and  promote healthy life choices', 'The  state  should  have no  role  in  encouraging or  discouraging  particular lifestyle choices'],
    'Religion': ['Religion creates a sense of  community  for  people,  and  organizes  human life', 'Religion    has    proven over the years that it is a  harmful,   destructive force'],
    'Right to privacy': ['The right to privacy is a fundamental right', 'Privacy   is   not   absolute. There   are   instances when it must be compromised  in  order to protect society'],
    'Self-determination': ['The political status of a territorial  entity  should be defined by its population', 'Self-determination cannot  be  handed  freely, especially not as a prize for violence'],
    'Sexual morality': ['Sexual   morality   must be protected by opposing  immoral  lifestyles, and not ignoring them', 'The   sexual   behaviors and preferences of individuals  are  private  and are  not  the  business  of the authorities'],
    'Subsidies': ['Providing   support   for [TOPIC]  would  benefit society, and is therefore  a  worthwhile  use of government money', 'There  are  better  ways to  make  use  of  public funds'],
    'Technology': ['[TOPIC]  is  better  than the older options', 'These new technologies are  not  as  reliable  as conventional ones'],
    'Tradition': ['A   society   should   respect  its  traditions  and try to avoid changes for the sake of change', 'Society   should   move on  with  the  times,  instead of clinging to old and  obsolete  traditions which are no longer relevant'],
    'Value of science': ['Theories which are not  based  on  scientific methods  should  not  be supported', 'Not  everything  can  be explained by science'],
    'Virtual life': ['The  virtual  world  enriches our lives in ways that other forms do not', 'In   the   virtual   world, people  lose  touch  with reality'],
    'Wealth distribution': ['Society  has  a  duty  to minimize inequality by allocating resources more evenly', 'The   way   to   achieve a   fair   distribution   of wealth is to let it be determined by the market forces'],
    'Welfare state': ['The state has a duty to provide  for  the  social and  economic  security of its citizens', 'State-sponsored welfare is counterproductive and actually exacerbates the problem']
}

In [9]:
for v in pas.values():
    v[0] = re.sub(' +', ' ', v[0])
    v[1] = re.sub(' +', ' ', v[1])


In [13]:
lst = []
for v in pas.values():
    lst.extend(v)

In [ ]:
pickle.dump(pas, open('PA_dict.pkl', 'wb'))
pickle.dump(lst, open('PA_dic.pkl', 'wb'))